In [2]:
import h2o
from h2o.estimators.gam import H2OGeneralizedAdditiveEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.13" 2021-10-19; OpenJDK Runtime Environment JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21); OpenJDK 64-Bit Server VM JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21, mixed mode)
  Starting server from /Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/19/s4zdyd_16mb8h239mgwds83m0000gn/T/tmpu1e8a7c9
  JVM stdout: /var/folders/19/s4zdyd_16mb8h239mgwds83m0000gn/T/tmpu1e8a7c9/h2o_liyanran_started_from_python.out
  JVM stderr: /var/folders/19/s4zdyd_16mb8h239mgwds83m0000gn/T/tmpu1e8a7c9/h2o_liyanran_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,"7 days, 23 hours and 43 minutes"
H2O_cluster_name:,H2O_from_python_liyanran_2ykx7o
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# create frame knots
knots1 = [-1.99905699, -0.98143075, 0.02599159, 1.00770987, 1.99942290]
frameKnots1 = h2o.H2OFrame(python_obj=knots1)
knots2 = [-1.999821861, -1.005257990, -0.006716042, 1.002197392, 1.999073589]
frameKnots2 = h2o.H2OFrame(python_obj=knots2)
knots3 = [-1.999675688, -0.979893796, 0.007573327,1.011437347, 1.999611676]
frameKnots3 = h2o.H2OFrame(python_obj=knots3)

In [11]:
# import the dataset
h2o_data = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/glm_test/multinomial_10_classes_10_cols_10000_Rows_train.csv")

# convert the C1, C2, and C11 columns to factors
h2o_data["C1"] = h2o_data["C1"].asfactor()
h2o_data["C2"] = h2o_data["C2"].asfactor()
h2o_data["C11"] = h2o_data["C11"].asfactor()

# split into train and validation sets
train, test = h2o_data.split_frame(ratios = [.8])

# set the predictor and response columns
y = "C11"
x = ["C1","C2"]

# specify the knots array
numKnots = [5,5,5]

# build the GAM model
h2o_model = H2OGeneralizedAdditiveEstimator(family='multinomial',
                                            gam_columns=["C6","C7","C8"],
                                            scale=[1,1,1],
                                            num_knots=numKnots,
                                            knot_ids=[frameKnots1.key, frameKnots2.key, frameKnots3.key])
h2o_model.train(x=x, y=y, training_frame=train)

# get the model coefficients
h2oCoeffs = h2o_model.coef()

# generate predictions using the test data
pred = h2o_model.predict(test)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gam Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gam prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [12]:
h2o_model

Model Details
=============
H2OGeneralizedAdditiveEstimator : Generalized Additive Model
Model Key: GAM_model_python_1667590787866_1


glm model summary: 
names    family       link         regularization                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
-------  -----------  -----------  ---------------------------------------------  ----------------------------  -----------------------------  ----------------------  ---------------------------------
         multinomial  multinomial  Elastic Net (alpha = 0.5, lambda = 9.269E-5 )  250                           234                            6                       _bdd43103aed2093f362afd9258694f64

ModelMetricsMultinomialGLM: gam
** Reported on train data. **

MSE: 0.45788731004121913
RMSE: 0.6766737101744231
LogLoss: 1.3169395244115436
Null degrees of freedom: 7975
Residual degrees of freedom: 7732
Null deviance: 32854.1633457294
Residual deviance: 21007.819293412944
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0    1    2     3     4    5    6    7    8     9     Error     Rate
---  ---  ----  ----  ---  ---  ---  ---  ----  ----  --------  -------------
67   27   6     80    2    0    65   2    120   5     0.820856  307 / 374
23   205  126   98    2    0    156  0    35    24    0.693572  464 / 669
2    72   587   121   0    19   128  4    82    75    0.461468  503 / 1,090
28   62   99    813   11   9    57   6    208   212   0.459801  692 / 1,505
6    4    10    103   13   1    4    0    78    24    0.946502  230 / 243
0    1    75    29    0    20   7    0    13    9     0.87013   134 / 154
23   99   103   49    0    3    475  4    49    33    0.433174  363 / 838
0    0    34    33    0    0    17   10   36    33    0.93865   153 / 163
46   23   65    186   13   3    41   5    975   200   0.373796  582 / 1,557
2    6    99    245   0    1    23   6    222   779   0.436732  604 / 1,383
197  499  1204  1757  41   56   973  37   1818  1394  0.505517  4,032 / 7,976

Top-10 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.494483
2    0.736835
3    0.861334
4    0.927031
5    0.961133
6    0.982949
7    0.992352
8    0.99674
9    0.999122
10   1

Variable Importances: 
variable       relative_importance    scaled_importance      percentage
-------------  ---------------------  ---------------------  ----------------------
C1.4           9.506973266601562      1.0                    0.08374626170877857
C2.3           9.266468048095703      0.9747022304826747     0.08162766808213227
C1.2           8.928357124328613      0.9391377122826615     0.07864927263340736
C2.5           7.801051139831543      0.8205609631024205     0.0687189131639827
C8_0_center_2  6.96945333480835       0.733088559246544      0.06139342633837249
C1.5           6.570053577423096      0.6910773169525992     0.057875141846512886
C2.0           6.457330703735352      0.6792204545709888     0.056882173946457566
C1.3           5.619228363037109      0.5910638649608618     0.04949938912161449
C8_0_center_3  5.294803142547607      0.5569388904403988     0.046641550074618385
C2.1           5.271616458892822      0.5544999771286047     0.04643730020212385
---            ---                    ---                    ---
C2.4           4.312331199645996      0.4535966472941936     0.03798702353452406
C6_0_center_2  4.114548683166504      0.4327927057101448     0.03624477119805217
C7_0_center_3  3.8060662746429443     0.40034469098738623    0.03352737126514973
C6_0_center_3  3.1458628177642822     0.3309005642012105     0.027711685249177057
C8_0_center_1  2.8188703060150146    